In [0]:
dbutils.widgets.text("Environment", "dev", "Set the curent environment/catalog name")
dbutils.widgets.text("RunType", "once", "Set once to run as batch job")
dbutils.widgets.text("ProcessingTime", "10 seconds", "Set the microbatch interval for streaming")

In [0]:
env = dbutils.widgets.get("Environment")
once = True if dbutils.widgets.get("RunType") == "once" else False
processing_time = dbutils.widgets.get("ProcessingTime")

if once:
    print(f"Starting processing in batch mode")
else:
    print(f"Strarting processing in streaming mode")

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", 4)
#spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", True)
#spark.conf.set("spark.databricks.delta.autoCompact.enabled", True)
#spark.conf.set("spark.sql.streaming.stateStore.providerClass", "com.databricks.sql.streaming.state.RocksDBStateStoreProvider")

In [0]:
%run ./02-setup

In [0]:
SH = SetupHandler(env)
#SH.cleanup()

In [0]:
setup_required = spark.sql(f"SHOW DATABASES IN {SH.catalog}").filter(f"databaseName = '{SH.db_name}'").count() != 1
if setup_required:
    SH.setup()
    #SH.validate()
else:
    spark.sql(f"USE {SH.catalog}.{SH.db_name}")


In [0]:
%run ./03-bronze

In [0]:
%run ./04-silver

In [0]:

BZ = Bronze(env)
SL = Silver(env)

In [0]:
BZ.consume(once, processing_time)
#SL.upsert(once, processing_time)

In [0]:
df = spark.read.table('dev.bookstore_db.bronze')

In [0]:
display(df)